
# Data Preprocessing
In this notebook we load and process the raw data to develop the final dataset for the IBM-Z Datathon. We make use of two main datasets for a list of all observed geoeffective CMEs from the post-SOHO era between 1996-2024, and two data sets for features and targets:

#### Geo-effective CMEs Targets:
- The [Richardson and Cane list](https://izw1.caltech.edu/ACE/ASC/DATA/level3/icmetable2.htm); a list of near-Earth CMEs from 1996-2024.
- The [George Mason University CME/ICME list](http://solar.gmu.edu/heliophysics/index.php/GMU_CME/ICME_List); a list of geoeffective CMEs from 2007-2017.

#### CME Features:
- The [SOHO-LASCO CME Catalogue](https://cdaw.gsfc.nasa.gov/CME_list/); a list of all CMEs observed from 1996-2024 containing information on physical quantities.
- [OMNIWeb Plus data](https://omniweb.gsfc.nasa.gov/); a list of features associated with the solar wind and sunspot numbers.

We will proceed by cleaning and combining the datasets to obtain and final set of features and targets. We will then explore the final dataset to make some conclusions.


## Cleaning the data:

In [53]:
# Importing libraries:
# For data manipulation
import pandas as pd
from datetime import timedelta

# For data visualisation
import matplotlib.pyplot as plt

In [42]:
# Adding filepaths as variables
cane_file_path = r"data/Raw Data/RichardsonCane.csv"
gmu_file_path = r"data/Raw Data/GMU.csv"
soho_file_path = r"data/Raw Data/SOHO_LASCO.csv"
omniweb_file_path = r"data/Raw Data/OMNIWeb.csv"


#### SOHO-LASCO Catalogue

We begin by loading in the SOHO-LASCO Catalogue to obtain the physical quantities for all CMEs. The original dataset had 11 total features. Most of the data was missing for the mass and kinetic energy hence these have been excluded. We have also excluded the second-order speeds as these are correlated with the linear speed. As a result this dataset contains the dates and times for each CME, together with five features:
- Central Position Angle in degrees.
- Angular Width in degrees.
- Linear Speed in km/s.
- Acceleration in km/s$^2$.
- Measurement Position Angle in degrees.


After inspecting the dataset, we will do the following:
- Convert all missing values labelled as "------" and "NaN" to `None`.
- Convert Central PA values labelled as "Halo" to 360.
- Reformat the Acceleration column by removing asterisks.
- Convert all columns to numeric.
- Remove CME data corresponding to an angular width below 90 degrees as it is known that these are not likely to be geoeffective.
- Replace labelled columns "Date" and "Time" with a single "Datetime" column.

In [43]:
# Reading the SOHO-LASCO Dataset
soho_df = pd.read_csv(soho_file_path)

# Replace all missing values ('------' and 'NaN') with None
soho_df.replace(['------', 'NaN'], None, inplace=True)

# Convert Angular Width values labelled as "Halo" to 360
soho_df['CentralPA'] = soho_df['CentralPA'].replace('Halo', 360)

# Remove asterisks from the Acceleration column
soho_df['Accel'] = soho_df['Accel'].astype(str).str.replace('*', '', regex=False)

# Convert all columns to numeric, except the first two (Date and Time)
cols_to_convert = soho_df.columns[2:]  # Keep first two columns (Date and Time) as object
soho_df[cols_to_convert] = soho_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')

# Remove rows where Central PA is below 90 degrees
soho_df = soho_df[soho_df['AngularWidth'] >= 90]

# Combine first two columns
soho_df['Datetime'] = pd.to_datetime(soho_df['Date'] + ' ' + soho_df['Time'], dayfirst=True) # Creating Datetime column
soho_df = soho_df.drop(soho_df.columns[[0, 1]], axis=1) # Drop the first two columns by index
last_column = soho_df.pop(soho_df.columns[-1])  # Pop the last column
soho_df.insert(0, last_column.name, last_column)  # Insert it at the front

#### OMNIWeb Plus Dataset
To obtain the final list of features, we must concatenate the SOHO-LASCO Catalogue with the OMNIWeb Plus Dataset. This dataset contains 12 features associated with the solar wind:

- The X-component of the magnetic field in nT.
- The Y-component of the magnetic field in nT.
- The Z-component of the magnetic field in nT.
- Plasma Temperature in Kelvin.
- Solar Proton Density n/cc.
- Flow Speed in km/s.
- Longitude Angle in degrees.
- Latitude Angle in degrees.
- Proton Density Ratio (unitless).
- Flow Pressure in nPa.
- Plasma Beta (unitless).
- Sunspot Number.

After taking a look at the OMNIWeb Plus Data, we next to the following to the dataset:
- Reformat first column to match the SOHO-LASCO data.
- Match times by averaging the 6-hour window after CME ejection.
- Concatenate both datasets to obtain final list of all 17 features.

In [44]:
# Reading the OMNIWeb Plus Dataset
omniweb_df = pd.read_csv(omniweb_file_path)

# Reformat Datetime column
omniweb_df['Datetime'] = pd.to_datetime(omniweb_df['Datetime'] + ':00', dayfirst=True)

replacement_values = [9999, 99, 999, 999.9, 9999999., 9999., 99.99, 9.999, 999.99, 999] # Removing unavailable data
omniweb_df.replace(replacement_values, None, inplace=True)

# Define a function to get the 6-hour averaged data after each CME
def get_solar_wind_average(cme_time, omniweb_data, window_hours=6):
    # Get the end time for the 6-hour window
    end_time = cme_time + pd.Timedelta(hours=window_hours)

    # Filter OMNIWeb data for the 6-hour window
    filtered_omniweb = omniweb_data[(omniweb_data['Datetime'] >= cme_time) & (omniweb_data['Datetime'] <= end_time)]

    # Calculate the average of all numerical columns in this window
    return filtered_omniweb.mean()

# Apply this function to each CME in the Cane dataset
averaged_solar_wind = soho_df['Datetime'].apply(get_solar_wind_average, omniweb_data=omniweb_df)

# Combine the averaged solar wind features with the original Cane dataset
combined_df = pd.concat([soho_df, averaged_solar_wind], axis=1)
combined_df.columns.values[6] = 'Datetime_2' # Renaming duplicated column

features_df  = combined_df.drop('Datetime_2', axis=1) # Removing duplicated Datetime column

,Datetime,CentralPA,AngularWidth,LinearSpeed,Accel,MPA,BX,BY,BZ,Plasma_Temp,Proton_Density,Plasma_Speed,Plasma_Long_Angle,Plasma_Lat_Angle,Alpha_Prot_Ratio,Flow_Pressure,Plasma_Beta,Sunspot_No
12,1996-02-02 23:00:47,180,119,80.0,1.8,164,4.150000,-0.116667,3.333333e-02,75709.500000,3.466667,493.666667,1.466667,-1.000000,0.016667,1.506667,1.785000,12.0
83,1996-04-29 14:38:48,360,360,65.0,NaN,149,-0.583333,0.300000,7.833333e-01,25115.666667,11.800000,366.500000,-0.083333,-0.066667,0.012500,2.780000,20.178333,0.0
85,1996-05-01 08:41:46,94,95,314.0,0.7,70,3.083333,-0.366667,-1.850372e-17,49811.000000,6.900000,406.166667,2.850000,-1.916667,0.028667,2.116667,4.000000,0.0
188,1996-06-18 17:28:50,84,95,64.0,-0.4,79,0.933333,-4.950000,-1.900000e+00,60472.500000,9.083333,388.000000,-1.900000,-2.700000,0.031667,2.596667,2.695000,14.0
285,1996-07-20 09:28:16,31,175,246.0,9.4,34,0.533333,3.500000,-1.283333e+00,102147.166667,7.866667,422.500000,1.100000,-1.450000,0.019000,2.518333,3.020000,0.0


#### Richardson and Cane Dataset
After manually removing irrelevant columns, the Cane dataset contain three column corresponding to the targets that will be used to train our models:
- LASCO CME Time.
- Disturbance Time.
- Dst Index measured in nT.


After inspection, we will do the following:
- Convert the first two column to the correct datetime format.
- Remove CMEs with a Dst index greater than -30 nT and label CMEs with a Dst index of less  as Geoeffective.
- Calculate the transit time as the difference between the LASCO CME time and the Disturbance time and convert to hours.


In [45]:
# Reading the Richardson-Cane Dataset
cane_df = pd.read_csv(cane_file_path)

# Reformatting the first two columns
cane_df['LASCO CME Y/M/D (UT)'] = pd.to_datetime(cane_df['LASCO CME Y/M/D (UT)'], format='mixed')
cane_df['Disturbance Y/M/D (UT)'] = pd.to_datetime(cane_df['Disturbance Y/M/D (UT)'], format='mixed')

# Dropping CMEs with a Dst index greater than -30 nT
cane_df = cane_df[cane_df['Dst (nT)'] <= -30]

# Calculating transit time
cane_df['TransitTime'] = cane_df['Disturbance Y/M/D (UT)'] - cane_df['LASCO CME Y/M/D (UT)']

# Dropping CMEs with a transit time of 0
cane_df = cane_df[cane_df['TransitTime'].dt.total_seconds() > 0]

# Converting transit time to hours as a float
cane_df['TransitTime'] = cane_df['TransitTime'].dt.total_seconds() / 3600

# Labelling and creating Geoeffective column (positive class labelled as 1)
cane_df['Geoeffective'] = 1
cane_df = cane_df.drop(cane_df.columns[[1, 2]], axis=1)

#### George Mason University Dataset
The GMU dataset contains similar information to the Cane dataset, however, the CMEs are recorded from 2007-2017. The formatting is inconsistent so we will do the following:
- Convert transit time column to delta-time format.
- Concert transit time to hours as a float.


In [46]:
# Reading the GMU dataset
gmu_df = pd.read_csv(gmu_file_path)

# Formating transit time column
gmu_df['Transit time'] = pd.to_timedelta(gmu_df['Transit time'])

# Converting transit time to hours
gmu_df['Transit time'] = gmu_df['Transit time'].dt.total_seconds() / 3600

In [47]:
# Saving cleaned .csv files
#gmu_df.to_csv(r"data/cleaned_gmu.csv", index=False)
#cane_df.to_csv(r"data/cleaned_cane.csv", index=False)

## Merging all Datasets:
To merge the features dataset with the Cane and GMU dataset we will do the following:
- Reformat all columns to maintain consistency.
- Concatenate the Cane and GMU datasets and remove duplicate events.
- Set an epsilon window to iterate over each geoeffective event and match with the LASCO datetimes.

In [48]:
# Convert the datetime columns to a consistent format for merging
features_df['Datetime'] = pd.to_datetime(features_df['Datetime'])
cane_df['LASCO CME Y/M/D (UT)'] = pd.to_datetime(cane_df['LASCO CME Y/M/D (UT)'])
gmu_df['CME in LASCO'] = pd.to_datetime(gmu_df['CME in LASCO'])

# Rename columns for consistency
cane_df.rename(columns={'LASCO CME Y/M/D (UT)': 'Datetime'}, inplace=True)
gmu_df.rename(columns={'CME in LASCO': 'Datetime'}, inplace=True)

# Merge the cleaned_cane and cleaned_gmu datasets together
geoeffective_df = pd.concat([cane_df, gmu_df])

# Remove duplicate CMEs from the combined geoeffective dataframe based on the datetime
geoeffective_df = geoeffective_df.drop_duplicates(subset='Datetime')

# Set an epsilon time window (e.g., +/- 0.5 hours)
epsilon = timedelta(hours=0.5)

# Create new columns for Geoeffective and TransitTime with default values
merged_df = features_df.copy()
merged_df['Geoeffective'] = 0
merged_df['TransitTime'] = None

# Iterate over each geoeffective event and match with the features within the epsilon window
for _, row in geoeffective_df.iterrows():
    cme_datetime = row['Datetime']

    # Find matches within the epsilon window
    mask = (merged_df['Datetime'] >= cme_datetime - epsilon) & (merged_df['Datetime'] <= cme_datetime + epsilon)
    merged_df.loc[mask, 'Geoeffective'] = 1
    merged_df.loc[mask, 'TransitTime'] = row['TransitTime']

# Saving the final dataset as .csv
#merged_df.to_csv(r"data/final_dataset.csv", index=False)

## Data Exploration and Visualisation
Let us now explore the data to make some conclusions and discuss its potential impact. We will do the following:
- Inspect the distribution of the positive and negative classes.
- Inspect the distribution of the regression target.
- Examine the impact of missing values.
- Visualise correlation between features with respect to each ML task.
- Inspect the feature space with dimensionality reduction techniques.
- Analyse the uncertainty in labelling

#### Inspecting the Target Distributions
We will begin by taking a look at the positive and negative classes.

In [55]:
# Counting the number of events in the positive and negative class
geoeffective_events_count = merged_df['Geoeffective'].sum()
nongeo_events_count = merged_df.shape[0] - geoeffective_events_count

print(f"The positive class has {geoeffective_events_count} events")
print(f"The negative class has {nongeo_events_count} events")

The positive class has 237 events
The negative class has 5636 events


In [ ]:
# Plotting the classes as a bar chart
plt.style.use(['science'])
plt.figure(figsize=[8, 5], dpi=500)
plt.title("Binary Classes", fontsize=20)
plt.xlabel("$x$", fontsize=18)
plt.ylabel("$U(x)$", fontsize=18)
plt.grid(True)
plt.plot(x,y)

In [ ]:
# Calculate the counts of each class
class_counts = df['Geoeffective'].value_counts()

# Create a bar chart
plt.figure(figsize=(6,4))
bars = plt.bar(class_counts.index, class_counts.values, color=['blue', 'orange'])

# Add labels on the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, int(yval), ha='center', va='bottom')

# Add titles and labels
plt.title('Class Distribution of Geoeffective CMEs')
plt.xlabel('Geoeffective (1 = Yes, 0 = No)')
plt.ylabel('Count')
plt.xticks([0, 1], ['Non-Geoeffective', 'Geoeffective'])
plt.show()

## Discussion and Conclusions
We have at last obtained the final dataset on which to train our models. There are some important things to note which are discussed as follows:

<strong style="font-size:18px;">Class Imbalance:</strong> One of the most prominent issues is the extreme imbalance between geoeffective and non-geoeffective CMEs. We note that only about 4% of CMEs end up impacting Earth and causing geomagnetic storms. This skewed distribution will make it difficult for the models to effectively identify the minority class.

<strong style="font-size:18px;">Feature Selection:</strong> The success of CME prediction models heavily relies on the selection and representation of input features. Only solar onset (solar wind) parameters have been used, which limits the model’s ability to account for CME propagation in interplanetary space.

<strong style="font-size:18px;">Missing Values:</strong> Our final data set suffers from missing values. Eliminating data with missing values can further reduce the already small positive class size, making it even harder to train effective models. We will deal with this in the subsequent section.

<strong style="font-size:18px;">Class Overlap:</strong> There is substantial similarity between the features of geoeffective and non-geoeffective CMEs, leading to overlapping data points in feature space. This class overlap makes it hard for models to distinguish between the two classes and exacerbates misclassifications.

<strong style="font-size:18px;">Labelling Uncertainty:</strong> Due to the uncertain association between CMEs and geomagnetic storms, the models may sometimes be trained on inaccurate data. For instance, some storms might be caused by CMEs that are difficult to link definitively to solar eruptions, leading to uncertainty in labels. Another important note is that the data does not distinguish between front-facing and back-facing CMEs (i.e. CMEs facing towards Earth or away).
